In [9]:
!pip install dotenv

In [8]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_api'.")

# Configurer le module avec la clé API
genai.configure(api_key=api_key)

# Exemple d'utilisation de l'API
for m in genai.list_models():
    print(m)


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      max_temperature=None,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko

In [29]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import json
import glob
import re
import time

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

api_key = os.environ.get("cle_gemini_api")
if not api_key:
    raise ValueError("La clé API n'est pas définie dans le fichier .env. Veuillez configurer la variable 'cle_api'.")

# Configurer le module avec la clé API
genai.configure(api_key=api_key)

# Sélectionner le modèle
model = genai.GenerativeModel('gemini-1.5-flash')


def load_chapters(json_file):
    """Charge les chapitres et sous-chapitres depuis un fichier JSON."""
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data


def extract_metadata_from_tex(tex_file):
    """Extrait les métadonnées spécifiques des fichiers LaTeX (titre, thème, etc.)"""
    try:
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraction des métadonnées courantes
        metadata = {}
        patterns = {
            "titre": r'\\titre\{([^}]*)\}',
            "theme": r'\\theme\{([^}]*)\}',
            "uuid": r'\\uuid\{([^}]*)\}',
            "auteur": r'\\auteur\{([^}]*)\}',
            "organisation": r'\\organisation\{([^}]*)\}'
        }
        
        for key, pattern in patterns.items():
            match = re.search(pattern, content)
            if match:
                metadata[key] = match.group(1)
        
        return metadata
    except Exception as e:
        print(f"Erreur lors de l'extraction des métadonnées de {tex_file}: {e}")
        return {}


def extract_exercise_content(tex_file):
    """Extrait le contenu pertinent d'un exercice mathématique."""
    try:
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Extraire le contenu entre \contenu{...}
        contenu_pattern = r'\\contenu\{(.*?)\}'
        contenu_match = re.search(contenu_pattern, content, re.DOTALL)
        
        if contenu_match:
            exercise_content = contenu_match.group(1)
        else:
            # Si pas de balise contenu, prendre tout le fichier
            exercise_content = content
        
        # Extraire les questions et réponses pour mieux comprendre le contexte
        question_pattern = r'\\question\{([^}]*)\}'
        reponse_pattern = r'\\reponse\{([^}]*)\}'
        
        questions = [m.group(1) for m in re.finditer(question_pattern, exercise_content)]
        reponses = [m.group(1) for m in re.finditer(reponse_pattern, exercise_content)]
        
        # Extraire les formules mathématiques (inline et display)
        math_pattern = r'\$(.*?)\$|\$\$(.*?)\$\$'
        math_matches = re.finditer(math_pattern, exercise_content, re.DOTALL)
        formulas = [m.group(1) or m.group(2) for m in math_matches if m.group(1) or m.group(2)]
        
        # Extraire le texte d'introduction (souvent présent dans \texte{...})
        texte_pattern = r'\\texte\{([^}]*)\}'
        texte_match = re.search(texte_pattern, exercise_content)
        intro_text = texte_match.group(1) if texte_match else ""
        
        # Construire un résumé
        summary = {
            "intro": intro_text,
            "questions": questions,
            "reponses": reponses,
            "nb_formules": len(formulas),
            "formules": formulas[:5]  # Garder seulement quelques formules représentatives
        }
        
        return exercise_content, summary
    except Exception as e:
        print(f"Erreur lors de l'extraction du contenu de l'exercice {tex_file}: {e}")
        return "", {}


def classify_tex_with_metadata_first(tex_file, chapters):
    """Stratégie de classification qui privilégie les métadonnées avant tout."""
    print(f"Classification du fichier: {tex_file}")
    
    # 1. Extraire les métadonnées
    metadata = extract_metadata_from_tex(tex_file)
    
    if metadata:
        print(f"  Métadonnées trouvées: {metadata}")
        
        # Si un thème est présent, c'est notre information la plus précieuse
        if "theme" in metadata and metadata["theme"]:
            theme = metadata["theme"]
            print(f"  Thème trouvé: {theme}")
            
            # Construire un prompt spécifique basé sur le thème
            prompt = f"""
            Vous êtes un expert en classification de documents mathématiques. Votre tâche est de déterminer
            le chapitre et le sous-chapitre les plus pertinents pour un exercice mathématique.
            
            Les informations suivantes sont disponibles:
            
            TITRE DE L'EXERCICE: {metadata.get("titre", "Non spécifié")}
            THÈME DE L'EXERCICE: {theme}
            
            Voici une liste de chapitres et sous-chapitres disponibles:
            {json.dumps(chapters, indent=2, ensure_ascii=False)}
            
            Basez-vous principalement sur les mots-clés présents dans le thème pour déterminer le chapitre
            et le sous-chapitre les plus appropriés.
            
            Fournissez votre réponse UNIQUEMENT au format JSON suivant:
            {{
                "chapitre": "Nom du chapitre",
                "sousChapitre": "Nom du sous-chapitre"
            }}
            """
            
            try:
                response = model.generate_content(
                    prompt,
                    generation_config=genai.GenerationConfig(
                        temperature=0.1,
                        top_p=0.95,
                        top_k=40
                    )
                )
                
                response_text = response.text.strip()
                
                # Extraire le JSON 
                json_pattern = r'\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*"chapitre"\s*:\s*"[^"]*"\s*,\s*"sousChapitre"\s*:\s*"[^"]*"(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\}'
                match = re.search(json_pattern, response_text)
                
                if match:
                    json_string = match.group(0)
                    json_string = json_string.replace("`", "")
                    
                    try:
                        classification = json.loads(json_string)
                        if "chapitre" in classification and "sousChapitre" in classification:
                            print(f"  Classification basée sur le thème: {classification['chapitre']} / {classification['sousChapitre']}")
                            return classification["chapitre"], classification["sousChapitre"]
                    except json.JSONDecodeError as e:
                        print(f"  Erreur lors du décodage JSON: {e}")
                
                # Méthode alternative si l'extraction JSON échoue
                chapitre_pattern = r'"chapitre"\s*:\s*"([^"]*)"'
                sousChapitre_pattern = r'"sousChapitre"\s*:\s*"([^"]*)"'
                
                chapitre_match = re.search(chapitre_pattern, response_text)
                sous_chapitre_match = re.search(sousChapitre_pattern, response_text)
                
                if chapitre_match and sous_chapitre_match:
                    chapitre = chapitre_match.group(1)
                    sous_chapitre = sous_chapitre_match.group(1)
                    print(f"  Classification basée sur le thème (méthode alternative): {chapitre} / {sous_chapitre}")
                    return chapitre, sous_chapitre
                    
            except Exception as e:
                print(f"  Erreur lors de l'appel à l'API: {e}")
    
    # 2. Si les métadonnées ne suffisent pas, extraire et analyser le contenu de l'exercice
    print("  Tentative de classification basée sur le contenu de l'exercice...")
    exercise_content, summary = extract_exercise_content(tex_file)
    
    # Construire un nouveau prompt basé sur le contenu de l'exercice
    if exercise_content:
        prompt = f"""
        Vous êtes un expert en classification d'exercices mathématiques. Votre tâche est de déterminer
        le chapitre et le sous-chapitre les plus pertinents pour cet exercice.
        
        TITRE: {metadata.get("titre", "Non spécifié")}
        
        INTRODUCTION DE L'EXERCICE:
        {summary.get("intro", "Non disponible")}
        
        QUESTIONS:
        {json.dumps(summary.get("questions", []), indent=2, ensure_ascii=False)[:1000]}
        
        QUELQUES FORMULES MATHÉMATIQUES:
        {json.dumps(summary.get("formules", []), indent=2, ensure_ascii=False)[:1000]}
        
        Voici une liste de chapitres et sous-chapitres disponibles:
        {json.dumps(chapters, indent=2, ensure_ascii=False)}
        
        Identifiez les concepts mathématiques abordés dans cet exercice et déterminez le chapitre et
        le sous-chapitre les plus appropriés.
        
        Fournissez votre réponse UNIQUEMENT au format JSON suivant:
        {{
            "chapitre": "Nom du chapitre",
            "sousChapitre": "Nom du sous-chapitre"
        }}
        """
        
        try:
            response = model.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(
                    temperature=0.1,
                    top_p=0.95,
                    top_k=40
                )
            )
            
            response_text = response.text.strip()
            
            # Extraire le JSON
            json_pattern = r'\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*"chapitre"\s*:\s*"[^"]*"\s*,\s*"sousChapitre"\s*:\s*"[^"]*"(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\}'
            match = re.search(json_pattern, response_text)
            
            if match:
                json_string = match.group(0)
                json_string = json_string.replace("`", "")
                
                try:
                    classification = json.loads(json_string)
                    if "chapitre" in classification and "sousChapitre" in classification:
                        print(f"  Classification basée sur le contenu: {classification['chapitre']} / {classification['sousChapitre']}")
                        return classification["chapitre"], classification["sousChapitre"]
                except json.JSONDecodeError as e:
                    print(f"  Erreur lors du décodage JSON: {e}")
            
            # Méthode alternative si l'extraction JSON échoue
            chapitre_pattern = r'"chapitre"\s*:\s*"([^"]*)"'
            sousChapitre_pattern = r'"sousChapitre"\s*:\s*"([^"]*)"'
            
            chapitre_match = re.search(chapitre_pattern, response_text)
            sous_chapitre_match = re.search(sousChapitre_pattern, response_text)
            
            if chapitre_match and sous_chapitre_match:
                chapitre = chapitre_match.group(1)
                sous_chapitre = sous_chapitre_match.group(1)
                print(f"  Classification basée sur le contenu (méthode alternative): {chapitre} / {sous_chapitre}")
                return chapitre, sous_chapitre
                
        except Exception as e:
            print(f"  Erreur lors de l'appel à l'API: {e}")
    
    # 3. Si tout échoue, tenter une dernière approche avec tout le contenu brut
    print("  Tentative de classification avec le contenu brut...")
    return classify_raw_tex_file(tex_file, chapters)


def classify_raw_tex_file(tex_file, chapters):
    """
    Classifie directement un fichier .tex brut et retourne le chapitre et sous-chapitre les plus pertinents.
    Version simplifiée qui se concentre sur l'essentiel.
    """
    try:
        # Lire le contenu brut du fichier
        with open(tex_file, 'r', encoding='utf-8') as f:
            raw_content = f.read()
        
        # Si le fichier est très long, prendre un échantillon représentatif
        if len(raw_content) > 20000:
            # Extraire le début (contient souvent des métadonnées importantes)
            start = raw_content[:3000]
            
            # Extraire quelques sections du milieu (contenu principal)
            middle_start = len(raw_content) // 2 - 3000
            middle = raw_content[middle_start:middle_start + 6000]
            
            # Extraire la fin (peut contenir des conclusions)
            end = raw_content[-3000:]
            
            # Combiner les parties avec des séparateurs
            sample_content = f"{start}\n\n[...CONTENU INTERMÉDIAIRE OMIS...]\n\n{middle}\n\n[...CONTENU INTERMÉDIAIRE OMIS...]\n\n{end}"
            
            content_for_classification = sample_content
            print(f"  Fichier volumineux ({len(raw_content)} caractères), utilisation d'un échantillon.")
        else:
            content_for_classification = raw_content
        
        # Prompt spécifique pour les exercices mathématiques
        prompt = f"""
        Vous êtes un expert en classification d'exercices mathématiques.

        Votre tâche est de déterminer le chapitre et le sous-chapitre les plus pertinents pour cet exercice mathématique.
        
        Analysez le code LaTeX brut suivant, en prenant en compte:
        1. Le titre et les métadonnées de l'exercice (spécialement le champ "theme" s'il existe)
        2. Les formules mathématiques présentes dans l'exercice
        3. Les questions posées et les réponses attendues
        4. Le contexte de l'exercice
        
        Voici une liste de chapitres et sous-chapitres possibles:
        {json.dumps(chapters, indent=2, ensure_ascii=False)}

        Document LaTeX à classifier:
        ```latex
        {content_for_classification}
        ```

        Fournissez votre réponse UNIQUEMENT au format JSON suivant:
        {{
            "chapitre": "Nom du chapitre",
            "sousChapitre": "Nom du sous-chapitre"
        }}
        
        Ne fournissez que le JSON sans autre texte.
        """

        max_retries = 3
        delay = 40  # secondes

        for attempt in range(max_retries):
            try:
                response = model.generate_content(
                    prompt,
                    generation_config=genai.GenerationConfig(
                        temperature=0.1,
                        top_p=0.95,
                        top_k=40
                    )
                )
                
                response_text = response.text.strip()
                
                # Utiliser une expression régulière pour trouver un objet JSON
                json_pattern = r'\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*"chapitre"\s*:\s*"[^"]*"\s*,\s*"sousChapitre"\s*:\s*"[^"]*"(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\}'
                
                match = re.search(json_pattern, response_text)
                
                if match:
                    json_string = match.group(0)
                    json_string = json_string.replace("`", "")
                    
                    try:
                        classification = json.loads(json_string)
                        if "chapitre" in classification and "sousChapitre" in classification:
                            print(f"  Classification du contenu brut réussie: {classification['chapitre']} / {classification['sousChapitre']}")
                            return classification["chapitre"], classification["sousChapitre"]
                    except json.JSONDecodeError as e:
                        print(f"  Erreur de décodage JSON: {e}")
                else:
                    # Méthode alternative
                    chapitre_pattern = r'"chapitre"\s*:\s*"([^"]*)"'
                    sousChapitre_pattern = r'"sousChapitre"\s*:\s*"([^"]*)"'
                    
                    chapitre_match = re.search(chapitre_pattern, response_text)
                    sous_chapitre_match = re.search(sousChapitre_pattern, response_text)
                    
                    if chapitre_match and sous_chapitre_match:
                        chapitre = chapitre_match.group(1)
                        sous_chapitre = sous_chapitre_match.group(1)
                        print(f"  Classification du contenu brut réussie (méthode alternative): {chapitre} / {sous_chapitre}")
                        return chapitre, sous_chapitre

            except Exception as e:
                print(f"  Erreur lors de l'appel à l'API (tentative {attempt + 1}/{max_retries}): {e}")
                if "429" in str(e):
                    print(f"  Quota dépassé. Attente de {delay} secondes...")
                    time.sleep(delay)
                    delay *= 2
                else:
                    return None, None

        print(f"  Échec après {max_retries} tentatives.")
        return None, None
    except Exception as e:
        print(f"  Erreur lors du traitement du fichier {tex_file}: {e}")
        return None, None


def check_chapter_info_exists(tex_file):
    """Vérifie si les commandes \\chapitre et \\sousChapitre sont déjà présentes dans le fichier .tex."""
    try:
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
            # Recherche des patterns pour \chapitre{...} et \sousChapitre{...}
            chapitre_pattern = r'\\chapitre\{([^}]*)\}'
            sous_chapitre_pattern = r'\\sousChapitre\{([^}]*)\}'
            
            chapitre_match = re.search(chapitre_pattern, content)
            sous_chapitre_match = re.search(sous_chapitre_pattern, content)
            
            if chapitre_match and sous_chapitre_match:
                # Les deux commandes existent déjà
                return True, chapitre_match.group(1), sous_chapitre_match.group(1)
            elif chapitre_match:
                # Seulement \chapitre existe
                return False, chapitre_match.group(1), None
            elif sous_chapitre_match:
                # Seulement \sousChapitre existe
                return False, None, sous_chapitre_match.group(1)
            else:
                # Aucune des commandes n'existe
                return False, None, None
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {tex_file}: {e}")
        return False, None, None


def insert_chapter_info(tex_file, chapitre, sous_chapitre):
    """Insère ou met à jour les commandes \\chapitre et \\sousChapitre au début du fichier .tex."""
    try:
        # Vérifier si les informations de chapitre sont déjà présentes
        exists, existing_chapitre, existing_sous_chapitre = check_chapter_info_exists(tex_file)
        
        if exists:
            print(f"  Les informations de chapitre sont déjà présentes dans {tex_file}")
            print(f"  Chapitre existant: {existing_chapitre}")
            print(f"  Sous-chapitre existant: {existing_sous_chapitre}")
            return True
        
        # Lire le contenu actuel
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Si l'un des éléments existe déjà, le supprimer
        if existing_chapitre or existing_sous_chapitre:
            # Supprimer les lignes existantes
            content = re.sub(r'\\chapitre\{[^}]*\}\s*', '', content)
            content = re.sub(r'\\sousChapitre\{[^}]*\}\s*', '', content)
        
        # Réécrire le fichier avec les nouvelles informations au début
        with open(tex_file, 'w', encoding='utf-8') as f:
            f.write(f"\\chapitre{{{chapitre}}}\n")
            f.write(f"\\sousChapitre{{{sous_chapitre}}}\n")
            f.write(content)
        
        return True
    except Exception as e:
        print(f"Erreur lors de l'écriture dans le fichier {tex_file}: {e}")
        return False


def main():
    """Fonction principale pour classifier et insérer les informations dans les fichiers .tex."""
    chapters = load_chapters("../metadata/exo7/chapitres_complet.json")
    tex_files = glob.glob("../src/latex/amscc/*.tex")
    
    # Pour les tests, limiter le nombre de fichiers
    # tex_files = tex_files[:5]  # Décommentez pour tester sur un petit échantillon
    
    total_files = len(tex_files)
    already_classified = 0
    newly_classified = 0
    failed_classification = 0

    print(f"=== DÉBUT DE LA CLASSIFICATION ({total_files} fichiers) ===\n")

    for i, tex_file in enumerate(tex_files):
        print(f"[{i+1}/{total_files}] Traitement du fichier: {tex_file}")
        
        # Vérifier si les informations de chapitre existent déjà
        exists, existing_chapitre, existing_sous_chapitre = check_chapter_info_exists(tex_file)
        
        if exists:
            print(f"  Fichier déjà classifié: {existing_chapitre} / {existing_sous_chapitre}")
            already_classified += 1
            continue
        
        # Utiliser la stratégie optimisée pour les exercices
        chapitre, sous_chapitre = classify_tex_with_metadata_first(tex_file, chapters)

        if chapitre and sous_chapitre:
            print(f"  Classification réussie: {chapitre} / {sous_chapitre}")
            success = insert_chapter_info(tex_file, chapitre, sous_chapitre)
            if success:
                print(f"  ✓ Informations insérées dans {tex_file}")
                newly_classified += 1
            else:
                print(f"  ✗ Échec de l'insertion des informations dans {tex_file}")
                failed_classification += 1
        else:
            print(f"  ✗ Impossible de classifier {tex_file}")
            failed_classification += 1
        
        # Ajouter une pause entre les fichiers pour éviter le rate limiting de l'API
        if i < total_files - 1:
            time.sleep(10)  # Pause d'une seconde
    
    # Afficher un résumé
    print("\n=== RÉSUMÉ DU TRAITEMENT ===")
    print(f"Total des fichiers traités: {total_files}")
    print(f"Fichiers déjà classifiés: {already_classified}")
    print(f"Fichiers nouvellement classifiés: {newly_classified}")
    print(f"Échecs de classification: {failed_classification}")
    
    if total_files - already_classified > 0:
        success_rate = (newly_classified / (total_files - already_classified)) * 100
        print(f"Taux de réussite: {success_rate:.2f}% des fichiers non classifiés")


if __name__ == "__main__":
    main()

=== DÉBUT DE LA CLASSIFICATION (458 fichiers) ===

[1/458] Traitement du fichier: ../src/latex/amscc/YBwt.tex
  Fichier déjà classifié: Probabilité discrète / Lois de distributions
[2/458] Traitement du fichier: ../src/latex/amscc/trhY.tex
  Fichier déjà classifié: Statistique / Tests d'hypothèses, intervalle de confiance
[3/458] Traitement du fichier: ../src/latex/amscc/aWAS.tex
  Fichier déjà classifié: Développement limité / Calculs
[4/458] Traitement du fichier: ../src/latex/amscc/DEZs.tex
  Fichier déjà classifié: Statistique / Tests d'hypothèses, intervalle de confiance
[5/458] Traitement du fichier: ../src/latex/amscc/5cbY.tex
  Fichier déjà classifié: Statistique / Tests d'hypothèses, intervalle de confiance
[6/458] Traitement du fichier: ../src/latex/amscc/ouYw.tex
  Fichier déjà classifié: Calcul d'intégrales / Autre
[7/458] Traitement du fichier: ../src/latex/amscc/6NIK.tex
  Fichier déjà classifié: Probabilité discrète / Lois de distributions
[8/458] Traitement du fichier: 